
<a id='optimization-solver-packages'></a>
<div id="qe-notebook-header" style="text-align:right;">
        <a href="https://quantecon.org/" title="quantecon.org">
                <img style="width:250px;display:inline;" src="https://assets.quantecon.org/img/qe-menubar-logo.svg" alt="QuantEcon">
        </a>
</div>

# Solvers, Optimizers, and Automatic Differentiation

## Contents

- [Solvers, Optimizers, and Automatic Differentiation](#Solvers,-Optimizers,-and-Automatic-Differentiation)  
  - [Overview](#Overview)  
  - [Introduction to Automatic Differentiation](#Introduction-to-Automatic-Differentiation)  
  - [Optimization](#Optimization)  
  - [Systems of Equations and Least Squares](#Systems-of-Equations-and-Least-Squares)  
  - [LeastSquaresOptim.jl](#LeastSquaresOptim.jl)  
  - [Additional Notes](#Additional-Notes)  

## Overview

In this lecture we introduce a few of the Julia libraries that we’ve found particularly useful for quantitative work in economics

### Setup

In [1]:
using InstantiateFromURL
github_project("QuantEcon/quantecon-notebooks-jl", path = "more_julia", version = "v0.9.10");
# ] instantiate  # uncomment if required

In [2]:
using LinearAlgebra, Statistics, Compat
using ForwardDiff, Flux, Optim, JuMP, Ipopt, BlackBoxOptim, Roots, NLsolve
using LeastSquaresOptim, Flux.Tracker
using Flux.Tracker: update!
using Optim: converged, maximum, maximizer, minimizer, iterations #some extra functions

## Introduction to Automatic Differentiation

Automatic differentiation (AD, sometimes called algorithmic differentiation) is a crucial way to increase the performance of both estimation and solution methods

There are essentially four ways to calculate the gradient or Jacobian on a computer

- Calculation by hand  
  
  - Where possible, you can calculate the derivative on “pen and paper” and potentially simplify the expression  
  - Sometimes, though not always, the most accurate and fastest option if there are algebraic simplifications  
  - The algebra is error prone for non-trivial setups  
  
- Finite differences  
  
  - Evaluate the function at least $ N $ times to get the gradient – Jacobians are even worse  
  - Large $ \Delta $ is numerically stable but inaccurate, too small of $ \Delta $ is numerically unstable but more accurate  
  - Avoid if you can, and use packages (e.g. [DiffEqDiffTools.jl](https://github.com/JuliaDiffEq/DiffEqDiffTools.jl) ) to get a good choice of $ \Delta $  
  


$$
\partial_{x_i}f(x_1,\ldots x_N) \approx \frac{f(x_1,\ldots x_i + \Delta,\ldots x_N) - f(x_1,\ldots x_i,\ldots x_N)}{\Delta}
$$

- Symbolic differentiation  
  
  - If you put in an expression for a function, some packages will do symbolic differentiation  
  - In effect, repeated applications of the chain rule, product rule, etc.  
  - Sometimes a good solution, if the package can handle your functions  
  
- Automatic Differentiation  
  
  - Essentially the same as symbolic differentiation, just occurring at a different time in the compilation process  
  - Equivalent to analytical derivatives since it uses the chain rule, etc.  
  


We will explore AD packages in Julia rather than the alternatives

### Automatic Differentiation

To summarize here, first recall the chain rule (adapted from [Wikipedia](https://en.wikipedia.org/wiki/Chain_rule))

$$
\frac{dy}{dx} = \frac{dy}{dw} \cdot \frac{dw}{dx}
$$

Consider functions composed of calculations with fundamental operations with known analytical derivatives, such as $ f(x_1, x_2) = x_1 x_2 + \sin(x_1) $

To compute $ \frac{d f(x_1,x_2)}{d x_1} $

$$
\begin{array}{l|l}
\text{Operations to compute value} &
\text{Operations to compute \$\frac{\partial f(x_1,x_2)}{\partial x_1}\$}
\\
\hline
w_1 = x_1 &
\frac{d w_1}{d x_1} = 1 \text{ (seed)}\\
w_2 = x_2 &
\frac{d  w_2}{d x_1} = 0 \text{ (seed)}
\\
w_3 = w_1 \cdot w_2 &
\frac{\partial  w_3}{\partial x_1} = w_2 \cdot \frac{d  w_1}{d x_1} + w_1 \cdot \frac{d  w_2}{d x_1}
\\
w_4 = \sin w_1 &
\frac{d  w_4}{d x_1} = \cos w_1 \cdot \frac{d  w_1}{d x_1}
\\
w_5 = w_3 + w_4 &
\frac{\partial  w_5}{\partial x_1} = \frac{\partial  w_3}{\partial x_1} + \frac{d  w_4}{d x_1}
\end{array}
$$

### Using Dual Numbers

One way to implement this (used in forward-mode AD) is to use [dual numbers](https://en.wikipedia.org/wiki/Dual_number)

Take a number $ x $ and augment it with an infinitesimal $ \epsilon $ such that $ \epsilon^2 = 0 $, i.e. $ x \to x + x' \epsilon $

All math is then done with this (mathematical, rather than Julia) tuple $ (x, x') $ where the $ x' $ may be hidden from the user

With this definition, we can write a general rule for differentiation of $ g(x,y) $ as

$$
g \big( \left(x,x'\right),\left(y,y'\right) \big) = \left(g(x,y),\partial_x g(x,y)x' + \partial_y g(x,y)y' \right)
$$

This calculation is simply the chain rule for the total derivative

An AD library using dual numbers concurrently calculates the function and its derivatives, repeating the chain rule until it hits a set of intrinsic rules such as

$$
\begin{align*}
x + y \to \left(x,x'\right) + \left(y,y'\right) &= \left(x + y,\underbrace{x' + y'}_{\partial(x + y) = \partial x + \partial y}\right)\\
x y \to \left(x,x'\right) \times \left(y,y'\right) &= \left(x y,\underbrace{x'y + y'x}_{\partial(x y) = y \partial x + x \partial y}\right)\\
\exp(x) \to \exp(\left(x, x'\right)) &= \left(\exp(x),\underbrace{x'\exp(x)}_{\partial(\exp(x)) = \exp(x)\partial x} \right)
\end{align*}
$$

### ForwardDiff.jl

We have already seen one of the AD packages in Julia

In [3]:
using ForwardDiff
h(x) = sin(x[1]) + x[1] * x[2] + sinh(x[1] * x[2]) # multivariate.
x = [1.4 2.2]
@show ForwardDiff.gradient(h,x) # use AD, seeds from x

#Or, can use complicated functions of many variables
f(x) = sum(sin, x) + prod(tan, x) * sum(sqrt, x)
g = (x) -> ForwardDiff.gradient(f, x); # g() is now the gradient
@show g(rand(20)); # gradient at a random point
# ForwardDiff.hessian(f,x') # or the hessian

ForwardDiff.gradient(h, x) = [26.3548 16.6631]
g(rand(20)) = 

[0.982778, 0.956764, 0.810784, 0.830806, 0.96744, 0.811213, 0.85708, 0.851495, 0.79613, 0.830734, 0.938146, 0.650291, 0.966342, 0.880611, 0.56282, 0.685821, 0.650282, 0.791788, 0.878932, 0.952101]


We can even auto-differentiate complicated functions with embedded iterations

In [4]:
function squareroot(x) #pretending we don't know sqrt()
    z = copy(x) # Initial starting point for Newton’s method
    while abs(z*z - x) > 1e-13
        z = z - (z*z-x)/(2z)
    end
    return z
end
squareroot(2.0)

1.4142135623730951

In [5]:
using ForwardDiff
dsqrt(x) = ForwardDiff.derivative(squareroot, x)
dsqrt(2.0)

0.35355339059327373

### Flux.jl

Another is [Flux.jl](https://github.com/FluxML/Flux.jl), a machine learning library in Julia

AD is one of the main reasons that machine learning has become so powerful in
recent years, and is an essential component of any machine learning package

In [6]:
using Flux
using Flux.Tracker
using Flux.Tracker: update!

f(x) = 3x^2 + 2x + 1

# df/dx = 6x + 2
df(x) = Tracker.gradient(f, x)[1]

df(2) # 14.0 (tracked)

14.0 (tracked)

In [7]:
A = rand(2,2)
f(x) = A * x
x0 = [0.1, 2.0]
f(x0)
Flux.jacobian(f, x0)

2×2 Array{Float64,2}:
 0.40994    0.685825
 0.0734767  0.64366 

As before, we can differentiate complicated functions

In [8]:
dsquareroot(x) = Tracker.gradient(squareroot, x)

dsquareroot (generic function with 1 method)

From the documentation, we can use a machine learning approach to a linear regression

In [9]:
W = rand(2, 5)
b = rand(2)

predict(x) = W*x .+ b

function loss(x, y)
ŷ = predict(x)
sum((y .- ŷ).^2)
end

x, y = rand(5), rand(2) # Dummy data
loss(x, y) # ~ 3

4.320238293564178

In [10]:
W = param(W)
b = param(b)

gs = Tracker.gradient(() -> loss(x, y), Params([W, b]))

Δ = gs[W]

# Update the parameter and reset the gradient
update!(W, -0.1Δ)

loss(x, y) # ~ 2.5

1.2877439620340179 (tracked)

## Optimization

There are a large number of packages intended to be used for optimization in Julia

Part of the reason for the diversity of options is that Julia makes it possible to efficiently implement a large number of variations on optimization routines

The other reason is that different types of optimization problems require different algorithms

### Optim.jl

A good pure-Julia solution for the (unconstrained or box-bounded) optimization of
univariate and multivariate function is the [Optim.jl](https://github.com/JuliaNLSolvers/Optim.jl) package

By default, the algorithms in `Optim.jl` target minimization rather than
maximization, so if a function is called `optimize` it will mean minimization

#### Univariate Functions on Bounded Intervals

[Univariate optimization](http://julianlsolvers.github.io/Optim.jl/stable/user/minimization/#minimizing-a-univariate-function-on-a-bounded-interval)
defaults to a robust hybrid optimization routine called [Brent’s method](https://en.wikipedia.org/wiki/Brent%27s_method)

In [11]:
using Optim
using Optim: converged, maximum, maximizer, minimizer, iterations #some extra functions

result = optimize(x-> x^2, -2.0, 1.0)

Results of Optimization Algorithm
 * Algorithm: Brent's Method
 * Search Interval: [-2.000000, 1.000000]
 * Minimizer: 0.000000e+00
 * Minimum: 0.000000e+00
 * Iterations: 5
 * Convergence: max(|x - x_upper|, |x - x_lower|) <= 2*(1.5e-08*|x|+2.2e-16): true
 * Objective Function Calls: 6

Always check if the results converged, and throw errors otherwise

In [12]:
converged(result) || error("Failed to converge in $(iterations(result)) iterations")
xmin = result.minimizer
result.minimum

0.0

The first line is a logical OR between `converged(result)` and `error("...")`

If the convergence check passes, the logical sentence is true, and it will proceed to the next line; if not, it will throw the error

Or to maximize

In [13]:
f(x) = -x^2
result = maximize(f, -2.0, 1.0)
converged(result) || error("Failed to converge in $(iterations(result)) iterations")
xmin = maximizer(result)
fmax = maximum(result)

-0.0

**Note:** Notice that we call `optimize` results using `result.minimizer`, and `maximize` results using `maximizer(result)`

#### Unconstrained Multivariate Optimization

There are a variety of [algorithms and options](http://julianlsolvers.github.io/Optim.jl/stable/user/minimization/#_top) for multivariate optimization

From the documentation, the simplest version is

In [14]:
f(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
x_iv = [0.0, 0.0]
results = optimize(f, x_iv) # i.e. optimize(f, x_iv, NelderMead())

 * Status: success

 * Candidate solution
    Minimizer: [1.00e+00, 1.00e+00]
    Minimum:   3.525527e-09

 * Found with
    Algorithm:     Nelder-Mead
    Initial Point: [0.00e+00, 0.00e+00]

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Iterations:    60
    f(x) calls:    118


The default algorithm in `NelderMead`, which is derivative-free and hence requires many function evaluations

To change the algorithm type to [L-BFGS](http://julianlsolvers.github.io/Optim.jl/stable/algo/lbfgs/)

In [15]:
results = optimize(f, x_iv, LBFGS())
println("minimum = $(results.minimum) with argmin = $(results.minimizer) in "*
"$(results.iterations) iterations")

minimum = 5.378388330692143e-17 with argmin = [1.0, 1.0] in 24 iterations


Note that this has fewer iterations

As no derivative was given, it used [finite differences](https://en.wikipedia.org/wiki/Finite_difference) to approximate the gradient of `f(x)`

However, since most of the algorithms require derivatives, you will often want to use auto differentiation or pass analytical gradients if possible

In [16]:
f(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
x_iv = [0.0, 0.0]
results = optimize(f, x_iv, LBFGS(), autodiff=:forward) # i.e. use ForwardDiff.jl
println("minimum = $(results.minimum) with argmin = $(results.minimizer) in "*
"$(results.iterations) iterations")

minimum = 5.191703158437428e-27 with argmin = [1.0, 1.0] in 24 iterations


Note that we did not need to use `ForwardDiff.jl` directly, as long as our `f(x)` function was written to be generic (see the [generic programming lecture](generic_programming.html) )

Alternatively, with an analytical gradient

In [17]:
f(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
x_iv = [0.0, 0.0]
function g!(G, x)
    G[1] = -2.0 * (1.0 - x[1]) - 400.0 * (x[2] - x[1]^2) * x[1]
    G[2] = 200.0 * (x[2] - x[1]^2)
end

results = optimize(f, g!, x0, LBFGS()) # or ConjugateGradient()
println("minimum = $(results.minimum) with argmin = $(results.minimizer) in "*
"$(results.iterations) iterations")

minimum = 4.474575116368833e-22 with argmin = [1.0, 1.0] in 21 iterations


For derivative-free methods, you can change the algorithm – and have no need to provide a gradient

In [18]:
f(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
x_iv = [0.0, 0.0]
results = optimize(f, x_iv, SimulatedAnnealing()) # or ParticleSwarm() or NelderMead()

 * Status: failure (reached maximum number of iterations) (line search failed)

 * Candidate solution
    Minimizer: [9.41e-01, 8.92e-01]
    Minimum:   8.348758e-03

 * Found with
    Algorithm:     Simulated Annealing
    Initial Point: [0.00e+00, 0.00e+00]

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Iterations:    1000
    f(x) calls:    1001


However, you will note that this did not converge, as stochastic methods typically require many more iterations as a tradeoff for their global-convergence properties

See the [maximum likelihood](http://julianlsolvers.github.io/Optim.jl/stable/examples/generated/maxlikenlm/)
example and the accompanying [Jupyter notebook](https://nbviewer.jupyter.org/github/JuliaNLSolvers/Optim.jl/blob/gh-pages/v0.15.3/examples/generated/maxlikenlm.ipynb)

### JuMP.jl

The [JuMP.jl](https://github.com/JuliaOpt/JuMP.jl) package is an ambitious implementation of a modelling language for optimization problems in Julia

In that sense, it is more like an AMPL (or Pyomo) built on top of the Julia
language with macros, and able to use a variety of different commerical and open source solvers

If you have a linear, quadratic, conic, mixed-integer linear, etc. problem then this will likely be the ideal “meta-package” for calling various solvers

For nonlinear problems, the modelling language may make things difficult for complicated functions (as it is not designed to be used as a general-purpose nonlinear optimizer)

See the [quick start guide](http://www.juliaopt.org/JuMP.jl/0.18/quickstart.html) for more details on all of the options

The following is an example of calling a linear objective with a nonlinear constraint (provided by an external function)

Here `Ipopt` stands for `Interior Point OPTimizer`, a [nonlinear solver](https://github.com/JuliaOpt/Ipopt.jl) in Julia

In [19]:
using JuMP, Ipopt
# solve
# max( x[1] + x[2] )
# st sqrt(x[1]^2 + x[2]^2) <= 1

function squareroot(x) # pretending we don't know sqrt()
    z = x # Initial starting point for Newton’s method
    while abs(z*z - x) > 1e-13
        z = z - (z*z-x)/(2z)
    end
    return z
end
m = Model(with_optimizer(Ipopt.Optimizer))
# need to register user defined functions for AD
JuMP.register(m,:squareroot, 1, squareroot, autodiff=true)

@variable(m, x[1:2], start=0.5) # start is the initial condition
@objective(m, Max, sum(x))
@NLconstraint(m, squareroot(x[1]^2+x[2]^2) <= 1)
@show JuMP.optimize!(m)

ErrorException: error compiling #optimize!#79: error compiling optimize!: error compiling addOption: could not load library "/Users/arnavsood/.julia/packages/Ipopt/Iu7vT/deps/usr/lib/libipopt.1.10.10.dylib"
dlopen(/Users/arnavsood/.julia/packages/Ipopt/Iu7vT/deps/usr/lib/libipopt.1.10.10.dylib, 1): Library not loaded: @rpath/libgfortran.4.dylib
  Referenced from: /Users/arnavsood/.julia/packages/Ipopt/Iu7vT/deps/usr/lib/libipopt.1.10.10.dylib
  Reason: image not found

And this is an example of a quadratic objective

In [20]:
# solve
# min (1-x)^2 + 100(y-x^2)^2)
# st x + y >= 10

using JuMP,Ipopt
m = Model(with_optimizer(Ipopt.Optimizer)) # settings for the solver
@variable(m, x, start = 0.0)
@variable(m, y, start = 0.0)

@NLobjective(m, Min, (1-x)^2 + 100(y-x^2)^2)

JuMP.optimize!(m)
println("x = ", value(x), " y = ", value(y))

# adding a (linear) constraint
@constraint(m, x + y == 10)
JuMP.optimize!(m)
println("x = ", value(x), " y = ", value(y))

ErrorException: error compiling #optimize!#79: error compiling optimize!: error compiling addOption: could not load library "/Users/arnavsood/.julia/packages/Ipopt/Iu7vT/deps/usr/lib/libipopt.1.10.10.dylib"
dlopen(/Users/arnavsood/.julia/packages/Ipopt/Iu7vT/deps/usr/lib/libipopt.1.10.10.dylib, 1): Library not loaded: @rpath/libgfortran.4.dylib
  Referenced from: /Users/arnavsood/.julia/packages/Ipopt/Iu7vT/deps/usr/lib/libipopt.1.10.10.dylib
  Reason: image not found

### BlackBoxOptim.jl

Another package for doing global optimization without derivatives is [BlackBoxOptim.jl](https://github.com/robertfeldt/BlackBoxOptim.jl)

To see an example from the documentation

In [21]:
using BlackBoxOptim

function rosenbrock2d(x)
return (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
end

results = bboptimize(rosenbrock2d; SearchRange = (-5.0, 5.0), NumDimensions = 2);

Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64},RadiusLimitedSelector,BlackBoxOptim.AdaptiveDiffEvoRandBin{3},RandomBound{RangePerDimSearchSpace}}
0.00 secs, 0 evals, 0 steps

Optimization stopped after 10001 steps and 0.0927879810333252 seconds


Termination reason: Max number of steps (10000) reached
Steps per second = 107783.35608367315
Function evals per second = 110068.13475477991
Improvements/step = 0.1898
Total function evaluations = 10213


Best candidate found: 

[1.0, 1.0]

Fitness: 0.000000000



An example for [parallel execution](https://github.com/robertfeldt/BlackBoxOptim.jl/blob/master/examples/rosenbrock_parallel.jl) of the objective is provided

## Systems of Equations and Least Squares

### Roots.jl

A root of a real function $ f $ on $ [a,b] $ is an $ x \in [a, b] $ such that $ f(x)=0 $

For example, if we plot the function


<a id='equation-root-f'></a>
$$
f(x) = \sin(4 (x - 1/4)) + x + x^{20} - 1 \tag{1}
$$

with $ x \in [0,1] $ we get

<img src="https://s3-ap-southeast-2.amazonaws.com/lectures.quantecon.org/jl/_static/figures/sine-screenshot-2.png" style="">

  
The unique root is approximately 0.408

The [Roots.jl](https://github.com/JuliaLang/Roots.jl) package offers `fzero()` to find roots

In [22]:
using Roots
f(x) = sin(4 * (x - 1/4)) + x + x^20 - 1
fzero(f, 0, 1)

0.40829350427936706

### NLsolve.jl

The [NLsolve.jl](https://github.com/JuliaNLSolvers/NLsolve.jl/) package provides functions to solve for multivariate systems of equations and fixed points

From the documentation, to solve for a system of equations without providing a Jacobian

In [23]:
using NLsolve

f(x) = [(x[1]+3)*(x[2]^3-7)+18
        sin(x[2]*exp(x[1])-1)] # returns an array

results = nlsolve(f, [ 0.1; 1.2])

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.1, 1.2]
 * Zero: [-2.7586e-16, 1.0]
 * Inf-norm of residuals: 0.000000
 * Iterations: 4
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 5
 * Jacobian Calls (df/dx): 5

In the above case, the algorithm used finite differences to calculate the Jacobian

Alternatively, if `f(x)` is written generically, you can use auto-differentiation with a single setting

In [24]:
results = nlsolve(f, [ 0.1; 1.2], autodiff=:forward)

println("converged=$(NLsolve.converged(results)) at root=$(results.zero) in "*
"$(results.iterations) iterations and $(results.f_calls) function calls")

converged=true at root=[-3.48755e-16, 1.0] in 4 iterations and 5 function calls


Providing a function which operates inplace (i.e. modifies an argument) may help performance for large systems of equations (and hurt it for small ones)

In [25]:
function f!(F, x) # modifies the first argument
    F[1] = (x[1]+3)*(x[2]^3-7)+18
    F[2] = sin(x[2]*exp(x[1])-1)
end

results = nlsolve(f!, [ 0.1; 1.2], autodiff=:forward)

println("converged=$(NLsolve.converged(results)) at root=$(results.zero) in "*
"$(results.iterations) iterations and $(results.f_calls) function calls")

converged=true at root=[-3.48755e-16, 1.0] in 4 iterations and 5 function calls


## LeastSquaresOptim.jl

Many optimization problems can be solved using linear or nonlinear least squares

Let $ x \in R^N $ and $ F(x) : R^N \to R^M $ with $ M \geq N $, then the nonlinear least squares problem is

$$
\min_x F(x)^T F(x)
$$

While $ F(x)^T F(x) \to R $, and hence this problem could technically use any nonlinear optimizer, it is useful to exploit the structure of the problem

In particular, the Jacobian of $ F(x) $, can be used to approximate the Hessian of the objective

As with most nonlinear optimization problems, the benefits will typically become evident only when analytical or automatic differentiation is possible

If $ M = N $ and we know a root $ F(x^*) = 0 $ to the system of equations exists, then NLS is the defacto method for solving large **systems of equations**

An implementation of NLS is given in [LeastSquaresOptim.jl](https://github.com/matthieugomez/LeastSquaresOptim.jl)

From the documentation

In [26]:
using LeastSquaresOptim
function rosenbrock(x)
    [1 - x[1], 100 * (x[2]-x[1]^2)]
end
LeastSquaresOptim.optimize(rosenbrock, zeros(2), Dogleg())

Results of Optimization Algorithm
 * Algorithm: Dogleg
 * Minimizer: [1.0,1.0]
 * Sum of squares at Minimum: 0.000000
 * Iterations: 51
 * Convergence: true
 * |x - x'| < 1.0e-08: false
 * |f(x) - f(x')| / |f(x)| < 1.0e-08: true
 * |g(x)| < 1.0e-08: false
 * Function Calls: 52
 * Gradient Calls: 36
 * Multiplication Calls: 159


**Note:** Because there is a name clash between `Optim.jl` and this package, to use both we need to qualify the use of the `optimize` function (i.e. `LeastSquaresOptim.optimize`)

Here, by default it will use AD with `ForwardDiff.jl` to calculate the Jacobian,
but you could also provide your own calculation of the Jacobian (analytical or using finite differences) and/or calculate the function inplace

In [27]:
function rosenbrock_f!(out, x)
    out[1] = 1 - x[1]
    out[2] = 100 * (x[2]-x[1]^2)
end
LeastSquaresOptim.optimize!(LeastSquaresProblem(x = zeros(2),
                                f! = rosenbrock_f!, output_length = 2))

# if you want to use gradient
function rosenbrock_g!(J, x)
    J[1, 1] = -1
    J[1, 2] = 0
    J[2, 1] = -200 * x[1]
    J[2, 2] = 100
end
LeastSquaresOptim.optimize!(LeastSquaresProblem(x = zeros(2),
                                f! = rosenbrock_f!, g! = rosenbrock_g!, output_length = 2))

Results of Optimization Algorithm
 * Algorithm: Dogleg
 * Minimizer: [1.0,1.0]
 * Sum of squares at Minimum: 0.000000
 * Iterations: 51
 * Convergence: true
 * |x - x'| < 1.0e-08: false
 * |f(x) - f(x')| / |f(x)| < 1.0e-08: true
 * |g(x)| < 1.0e-08: false
 * Function Calls: 52
 * Gradient Calls: 36
 * Multiplication Calls: 159


## Additional Notes

Watch [this video](https://www.youtube.com/watch?v=vAp6nUMrKYg&feature=youtu.be) from one of Julia’s creators on automatic differentiation